## Some different approaches to summarizing the researcher feed

[by academic unit](#academic)  
[by academic unit and employee levels](#academic_employee)  
[by academic unit and employee levels as json output](#academic_employee_json)  

In [2]:
import xml.etree.ElementTree as ET
#rootEMP = ET.parse('TEST_LibEmpResearcher.xml').getroot()
#rootAFF = ET.parse('TEST_LibAffResearcher.xml').getroot()
rootFeed = ET.parse('LibEmplBio_20190807.xml').getroot()

<a id="academic"></a>
## breakdown by academic unit

In [2]:
groups, group_counts = [], []
org_codes, org_names, org_counts = [], [], []
ngroups, ngroup_counts = [], []
ncnumbers = []
researchers = rootFeed.findall('user')
for researcher in researchers:
    
    # user_group
    group = researcher.find('user_group').text
    if (not group in groups):
        groups.append(group)
        group_counts.append(1)
    else:
        pos = groups.index(group)
        group_counts[pos] += 1
    
    # researcher_organization
    org_code = researcher.find('researcher').find('researcher_organization_affiliations').find('researcher_organization_affiliation').find('organization_code').text
    if (not org_code in org_codes):
        org_codes.append(org_code)
        org_names.append(researcher.find('researcher').find('researcher_organization_affiliations').find('researcher_organization_affiliation').find('organization_name').text)
        org_counts.append(1)
    else:
        pos = org_codes.index(org_code)
        org_counts[pos] += 1
        
    if (org_code is None):
        ncnumbers.append(researcher.find('primary_id').text)
        if (not group in ngroups):
            ngroups.append(group)
            ngroup_counts.append(1)
        else:
            pos = ngroups.index(group)
            ngroup_counts[pos] += 1

group_total = 0
print ('\nGROUPS')
print ('---------------------')
for i in range(len(groups)):
    print (group_counts[i],'\t',groups[i])
    group_total += group_counts[i]
print ('---------------------')
print (len(researchers),'\t TOTAL\t (',group_total,')')

org_total = 0
print ('\nORGANIZATIONS')
print ('---------------------')
for i in range(len(org_codes)):
    space = ''
    if (not org_codes[i] is None):
        if (len(org_codes[i]) < 6):
            space = '\t '
    else:
        space = '\t '
    print (org_counts[i],'\t',org_codes[i],'\t',space,org_names[i])
    org_total += org_counts[i]
print ('-----------------------------')
print (len(researchers),'\t TOTAL\t (',org_total,')')

none_total = 0
print ('\nNONE GROUPS')
print ('---------------------')
for i in range(len(ngroups)):
    print (ngroup_counts[i],'\t',ngroups[i])
    none_total += ngroup_counts[i]
print ('---------------------')
print (none_total,'\t','TOTAL')


GROUPS
---------------------
682 	 ADJUNCT
1107 	 FACULTY
1574 	 MEDFACULTY
201 	 MEDADJUNCT
173 	 LAWADJUNCT
94 	 LAWFACULTY
334 	 MEDSTAFF
127 	 STAFF
---------------------
4292 	 TOTAL	 ( 4292 )

ORGANIZATIONS
---------------------
116 	 ARCH 	 	  ARCHITECTURE Department
44 	 PDO 	 	  PROFESSIONAL DEVELOPMENT OFFICE Department
52 	 JOU 	 	  JOURNALISM & MEDIA MANAGEMENT Department
16 	 PHILO 	 	  PHILOSOPHY Department
28 	 FIN 	 	  FINANCE Department
59 	 PSYCOL 	  PSYCHOLOGY Department
131 	 SURG 	 	  SURGERY Department
18 	 MECHE 	 	  MECHANICAL & AEROSPACE ENGINEERING Department
50 	 AAH 	 	  ART & ART HISTORY Department
71 	 RADIO 	 	  RADIOLOGY Department
32 	 KINSP 	 	  KINESIOLOGY & SPORT SCIENCES Department
347 	 MED 	 	  MEDICINE Department
585 	 None 	 	  None
34 	 OBGYN 	 	  OBSTETRICS & GYNECOLOGY Department
146 	 PEDI 	 	  PEDIATRICS Department
26 	 HUMGEN 	  HUMAN GENETICS Department
24 	 PHYS 	 	  PHYSICS Department
15 	 PHARM 	 	  MOLECULAR & CELLULAR PHARMACOLOGY D

<a id="academic_employee"></a>
## breakdown by academic unit and employee grouping

In [58]:
levels = ['primary', 'secondary', 'tertiary']
groups, group_counts = [], []
org_codes, org_names, org_counts = { 'primary':[], 'secondary': [], 'tertiary': [] }, \
                                   { 'primary':[], 'secondary': [], 'tertiary': [] }, \
                                   { 'primary':[], 'secondary': [], 'tertiary': [] }
ngroups, ngroup_counts = { 'primary':[], 'secondary': [], 'tertiary': [] }, \
                         { 'primary':[], 'secondary': [], 'tertiary': [] }
ncnumbers = { 'primary':[], 'secondary': [], 'tertiary': [] }
researchers = rootFeed.findall('user')
for researcher in researchers:
    
    # user_group
    group = researcher.find('user_group').text
    if (not group in groups):
        groups.append(group)
        group_counts.append(1)
    else:
        pos = groups.index(group)
        group_counts[pos] += 1
    
    # researcher_organization
    affiliations = researcher.find('researcher').find('researcher_organization_affiliations')
    for (i, affiliation) in enumerate(affiliations):
        org_code = affiliation.find('organization_code').text
        if (org_code is None):
            org_code = 'NONE'
        if (not org_code in org_codes[levels[i]]):
            org_codes[levels[i]].append(org_code)
            org_names[levels[i]].append(affiliation.find('organization_name').text)
            org_counts[levels[i]].append(1)
        else:
            pos = org_codes[levels[i]].index(org_code)
            org_counts[levels[i]][pos] += 1
        
        if (org_code == 'NONE'):
            ncnumbers[levels[i]].append(researcher.find('primary_id').text)
            if (not group in ngroups[levels[i]]):
                ngroups[levels[i]].append(group)
                ngroup_counts[levels[i]].append(1)
            else:
                pos = ngroups[levels[i]].index(group)
                ngroup_counts[levels[i]][pos] += 1

group_total = 0
print ('\nGROUPS')
print ('---------------------')
for i in range(len(groups)):
    print (str(group_counts[i]) + ',' + groups[i])
    group_total += group_counts[i]
print ('---------------------')
print (len(researchers),'\t TOTAL\t (',group_total,')')

print ('\nORGANIZATIONS')
print ('---------------------')
for level in levels:
    org_total = 0
    print (level)
    for i in range(len(org_codes[level])):
        space = ''
        if (not org_codes[level][i] is None):
            if (len(org_codes[level][i]) < 6):
                space = ','
        else:
            space = ','
        print (str(org_counts[level][i]) + ',' + str(org_codes[level][i]) + ',' + str(org_names[level][i]))
        org_total += org_counts[level][i]
    print ('-----------------------------')
    print (len(researchers),'\t TOTAL\t (',org_total,')')
    print ()

print ('\nNONE GROUPS')
print ('---------------------')
for level in levels:
    none_total = 0
    print (level)
    for i in range(len(ngroups[level])):
        print (str(ngroup_counts[level][i]) + ',' + ngroups[level][i])
        none_total += ngroup_counts[level][i]
    print ('---------------------')
    print (none_total,'\t','TOTAL')
    print ()


GROUPS
---------------------
682,ADJUNCT
1107,FACULTY
1574,MEDFACULTY
201,MEDADJUNCT
173,LAWADJUNCT
94,LAWFACULTY
334,MEDSTAFF
127,STAFF
---------------------
4292 	 TOTAL	 ( 4292 )

ORGANIZATIONS
---------------------
primary
116,ARCH,ARCHITECTURE Department
44,PDO,PROFESSIONAL DEVELOPMENT OFFICE Department
52,JOU,JOURNALISM & MEDIA MANAGEMENT Department
16,PHILO,PHILOSOPHY Department
28,FIN,FINANCE Department
59,PSYCOL,PSYCHOLOGY Department
131,SURG,SURGERY Department
18,MECHE,MECHANICAL & AEROSPACE ENGINEERING Department
50,AAH,ART & ART HISTORY Department
71,RADIO,RADIOLOGY Department
32,KINSP,KINESIOLOGY & SPORT SCIENCES Department
347,MED,MEDICINE Department
585,NONE,None
34,OBGYN,OBSTETRICS & GYNECOLOGY Department
146,PEDI,PEDIATRICS Department
26,HUMGEN,HUMAN GENETICS Department
24,PHYS,PHYSICS Department
15,PHARM,MOLECULAR & CELLULAR PHARMACOLOGY Department
24,BME,BIOMEDICAL ENGINEERING Department
12,PHYSBIO,PHYSIOLOGY & BIOPHYSICS Department
15,STUMUS,STUDIO MUSIC & JAZZ Dep

<a id="academic_employee_json"></a>
## Breakdown by Acadmic Unit and Employee levels - output as json

In [62]:
import json
summary = {}
researchers = rootFeed.findall('user')
for researcher in researchers:
    
    # researcher_organization
    affiliations = researcher.find('researcher').find('researcher_organization_affiliations')
    for (i, affiliation) in enumerate(affiliations):
        org_code = affiliation.find('organization_code').text
        if (org_code is None):
            org_code = 'NONE'
        if i==0:
            check = list(summary.keys())
        if i==1:
            first_org_code = affiliations[0].find('organization_code').text
            if first_org_code is None:
                first_org_code = 'NONE'
            check = list(summary[first_org_code]['g3'].keys())
        if i==2:
            first_org_code = affiliations[0].find('organization_code').text
            if first_org_code is None:
                first_org_code = 'NONE'
            second_org_code = affiliations[1].find('organization_code').text
            if second_org_code is None:
                second_org_code = 'NONE'
            check = list(summary[first_org_code]\
                                ['g3'][second_org_code]\
                                ['g4'].keys())
        if (not org_code in check):
            if i==0:
                org_name = affiliation.find('organization_name').text
                if org_name is None:
                    org_name = 'NONE'
                summary[org_code] = {
                    "name": org_name,
                    "count": 1,
                    "g3": {}
                }
            if i==1:
                first_org_code = affiliations[0].find('organization_code').text
                if first_org_code is None:
                    first_org_code = 'NONE'
                summary[first_org_code]['g3']\
                       [org_code] = {
                    "count": 1,
                    "g4": {}
                }
            if i==2:
                first_org_code = affiliations[0].find('organization_code').text
                if first_org_code is None:
                    first_org_code = 'NONE'
                second_org_code = affiliations[1].find('organization_code').text
                if second_org_code is None:
                    second_org_code = 'NONE'
                summary[first_org_code]\
                       ['g3'][second_org_code]\
                       ['g4'][org_code] = {
                    "count": 1
                }
        else:
            pos = org_codes[levels[i]].index(org_code)
            org_counts[levels[i]][pos] += 1
            if i==0:
                summary[org_code]['count'] += 1
            if i==1:
                first_org_code = affiliations[0].find('organization_code').text
                if first_org_code is None:
                    first_org_code = 'NONE'
                summary[first_org_code]\
                       ['g3'][org_code]['count'] += 1
            if i==2:
                first_org_code = affiliations[0].find('organization_code').text
                if first_org_code is None:
                    first_org_code = 'NONE'
                second_org_code = affiliations[1].find('organization_code').text
                if second_org_code is None:
                    second_org_code = 'NONE'
                summary[first_org_code]\
                       ['g3'][second_org_code]\
                       ['g4'][org_code]['count'] += 1
                
print (json.dumps(summary))

{"ARCH": {"name": "ARCHITECTURE Department", "count": 116, "g3": {"School of Architecture": {"count": 116, "g4": {"School of Architecture": {"count": 106}, "School of Architecture - Urban & Community Design": {"count": 1}, "School of Architecture - Academic Resource Center": {"count": 1}, "School of Architecture - Real Estate & Urban": {"count": 2}, "School of Architecture - Academic Services": {"count": 5}, "School of Architecture - Students Design Build": {"count": 1}}}}}, "PDO": {"name": "PROFESSIONAL DEVELOPMENT OFFICE Department", "count": 44, "g3": {"DCIE": {"count": 44, "g4": {"DCIE - Office of Professional Advancement": {"count": 44}}}}}, "JOU": {"name": "JOURNALISM & MEDIA MANAGEMENT Department", "count": 52, "g3": {"School of Communication": {"count": 52, "g4": {"School of Communication - Journalism & Media Mgmt": {"count": 50}, "School of Communication": {"count": 1}, "School of Communication - Academic Affairs": {"count": 1}}}}}, "PHILO": {"name": "PHILOSOPHY Department", "

In [61]:
affiliations[0].find('organization_code').text

In [42]:
check

['ARCH',
 'PDO',
 'JOU',
 'PHILO',
 'FIN',
 'PSYCOL',
 'SURG',
 'MECHE',
 'AAH',
 'RADIO',
 'KINSP',
 'MED']

In [43]:
org_code

'Human Resources'

In [44]:
i

1

In [13]:
i

0